In [6]:
%pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys; sys.path.append("../")
from feature_extractor import FeatureExtractor
import torch
import numpy as np
import time
from cascade import CascadeClassifier
import cv2
import joblib


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
parent = '../hFeatures6/'
data_path = 'new_data6'

# Features Extraction

In [4]:
mean = 111.02404842061983
std = 56.926623499738575

In [5]:
def transform1(img):
  return img
  # img = (img - img.mean()) / (img.std() + 1e-8)
  # return cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [6]:

feature_extractor = FeatureExtractor(shape=(24, 24),
                                     percentile=100,
                                     batch_size=1000,
                                     all_features_file=parent + "all_features.npz",
                                     selected_features_file=parent + "selected_features.npz",
                                     labels_file=parent + "labels.npy",
                                     indecies_file=parent + "indecies.npy",)

In [7]:

X, y = feature_extractor.extractFeatures(pos_path='../'+data_path+'/train/face',
                                  neg_path='../'+data_path+'/train/non-face',
                                  transform=transform1,
                                  save_to_file=False)
print('done')


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [13]:
import gc
gc.collect()

61

In [27]:
# del feature_extractor.f2
# del feature_extractor.f3
# del feature_extractor.f4

In [14]:
# indecies, X = feature_extractor.selectPercentile(X, y )
# # y = np.load(parent + 'labels.npy')
# print(X.shape, y.shape)
# print('done')


Now selecting percentile features
(12792, 10854) (10854,)
done


In [30]:
# feature_extractor.f2, feature_extractor.f3, feature_extractor.f4 = feature_extractor.describe_features(feature_extractor.shape)

In [15]:
# temp = np.load('../hFeatures3/all_features.npz')['arr_0']
temp = X
temp

array([[ -99.,  -83.,  234., ..., -248.,  -33.,    0.],
       [ -40., -136.,  268., ..., -423.,  -58.,    0.],
       [  29.,  231., -165., ...,  126.,  -92.,    0.],
       ...,
       [  -5.,  -39.,  -59., ...,   11.,    0.,    0.],
       [ -44., -197., -185., ...,   15.,   -3.,    0.],
       [ -31., -128.,  -98., ...,    7.,   -2.,    0.]], dtype=float32)

In [16]:
print(temp.min())
print(temp.max())
print(temp[3])
print(temp[4])

-26151.0
29662.0
[  29.  -41.  325. ... -505.  -91.    0.]
[   0.  239. -207. ...  144. -121.    0.]


# Training

In [17]:
%rm StrongClassifier/lastSC.last
s_t = time.time()
cascadeClassifier= CascadeClassifier(X, y, batchsize=10000, verbose=True, layers=[200])
tr_acc = cascadeClassifier.train()

cascadeClassifier.save(parent + 'cascadeClassifier.pkl')

print('accuracy: ', tr_acc)
print('time taken: ', time.time() - s_t, 's')
print('done')

rm: cannot remove 'StrongClassifier/lastSC.last': No such file or directory
$$$$$$$ Training layer 1 / 1 $$$$$$$
Starting to choose classifier
Finished training weak classifier 1 / 200
Starting to choose classifier
Finished training weak classifier 2 / 200
Starting to choose classifier
Finished training weak classifier 3 / 200
Starting to choose classifier
Finished training weak classifier 4 / 200
Starting to choose classifier
Finished training weak classifier 5 / 200
Starting to choose classifier
Finished training weak classifier 6 / 200
Starting to choose classifier
Finished training weak classifier 7 / 200
Starting to choose classifier
Finished training weak classifier 8 / 200
Starting to choose classifier
Finished training weak classifier 9 / 200
Starting to choose classifier
Finished training weak classifier 10 / 200
Starting to choose classifier
Finished training weak classifier 11 / 200
Starting to choose classifier
Finished training weak classifier 12 / 200
Starting to choose c

In [ ]:
cascadeClassifier.save_joblib(parent + 'cascadeClassifier.joblib')

In [ ]:
joblib.dump(feature_extractor, parent + 'feature_extractor.joblib')

In [18]:
# import pickle as pkl
# with open(parent + 'cascadeClassifier.pkl', 'rb') as f:
#   cascadeClassifier = pkl.load(f)

# Testing

In [19]:
# accuracy imports
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [20]:
s_t = time.time()
t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='../'+data_path+'/test/face',
                                            neg_path='../'+data_path+'/test/non-face',
                                            cascadeClassifier=cascadeClassifier,
                                            transform=transform1,
                                            use_percentile=False,)
print(t_features.shape, t_labels.shape)
predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
print('time taken: ', time.time() - s_t, 's')

Now reading dataset...
Now extracting features from dataset...
(191, 2715) (2715,)
test accuracy:  0.9804788213627993
time taken:  1.289139986038208 s


In [21]:
# report
print('accuracy: ', accuracy_score(t_labels, predictions))
print('f1 score macro: ', f1_score(t_labels, predictions, average='macro'))
print('f1 score binary: ', f1_score(t_labels, predictions))
print('f1 score micro: ', f1_score(t_labels, predictions, average='micro'))
print('precision: ', precision_score(t_labels, predictions))
print('recall: ', recall_score(t_labels, predictions))
print('confusion matrix: \n', confusion_matrix(t_labels, predictions))
print('classification report: \n', classification_report(t_labels, predictions, digits=4))


accuracy:  0.9804788213627993
f1 score macro:  0.9780447356724715
f1 score binary:  0.9707344008834898
f1 score micro:  0.9804788213627993
precision:  0.9701986754966887
recall:  0.9712707182320443
confusion matrix: 
 [[1783   27]
 [  26  879]]
classification report: 
               precision    recall  f1-score   support

           0     0.9856    0.9851    0.9854      1810
           1     0.9702    0.9713    0.9707       905

    accuracy                         0.9805      2715
   macro avg     0.9779    0.9782    0.9780      2715
weighted avg     0.9805    0.9805    0.9805      2715



# Try full image

In [22]:
# !mkdir /content/drive/MyDrive/GP/GP_try/hFeatures5_3
# !cp -r ../hFeatures5_3/* /content/drive/MyDrive/GP/GP_try/hFeatures5_3

mkdir: cannot create directory ‘/content/drive/MyDrive/GP/GP_try/hFeatures5_3’: File exists


In [23]:
# !mkdir ../../lfw_all
# !cp /content/drive/MyDrive/GP/lfw_all/* ../../lfw_all/

In [24]:
import random
from detect_face import FaceDetector
from PIL import Image
import os
import matplotlib.pyplot as plt
lfw_all = '../../lfw_all/'
all_images_names = os.listdir(lfw_all)
# all_images_names = sorted(all_images_names)
random.shuffle(all_images_names)
all_images_names = [lfw_all + name for name in all_images_names]
all_images_names[0]

'../../lfw_all/Carlos_Vives_0003.jpg'

In [25]:
show_images = list(range(0, 20))
rectangles_only = True
# resize_to = (100, 100)
random.shuffle(all_images_names)

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [27]:
feature_extractor.verbose = False

In [28]:
faceDetector = FaceDetector(cascadeClassifier, \
                            feature_extractor,\

                            window_size=(19,19),\
                            scale_dist=1.1,
                            max_size=300,
                            stride=5,

                            device=device,
                            use_percentile=False,
                            makeθ0=False,

                            verbose=False,
                            normalize_subwindows=False,
                            report_time=True,
                            use_sklearn=False
                            )

In [29]:
for i in show_images:
  # >>>> Read image <<<<
  img = Image.open(all_images_names[i])
  print(img.mode)
  img = np.array(img)
  org_sh = img.shape
  if not rectangles_only:
    plt.imshow(img)
    plt.show()
  # get subwindows
  img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  print(img2.shape)

  # >>>> Normalize image <<<<
  # normalize image
  # img2 = (img2 - img2.mean()) / img2.std()
  # img2 = cv2.resize(img, resize_to)
  # print(img2.shape)

  # img2 = cv2.normalize(img2, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  # resize image
  # img2 = cv2.resize(img2, (570, 380))

  # show image
  if not rectangles_only:
    plt.imshow(img2, cmap='gray')
    plt.show()

  # +++++++++++++++++++++
  # find face
  # >>>> Find face <<<<
  face_coordinates, region_max_conf, max_conf , timing=\
                  faceDetector.find_face(img2, \
                                         predict=True,
                                         confidence=True)
  print(timing)
  # region_max_conf *= img.shape[0] / resize_to[0]
  # ++++++++++++++++++++
  # rectangles
  # >>>> Draw rectangles <<<<
  tmp_img = img.copy()
  for face_coordinate in face_coordinates:
      for x1, y1, x2, y2 in face_coordinate:
          cv2.rectangle(tmp_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
  # if not rectangles_only:
  plt.imshow(tmp_img)
  plt.show()

  # >>>> Draw rectangles with max confidence <<<<
  # amx confidence
  tmp_img = img.copy()
  x1, y1, x2, y2 = region_max_conf
  cv2.rectangle(tmp_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
  plt.imshow(tmp_img)
  plt.show()



Output hidden; open in https://colab.research.google.com to view.